In [14]:
import geopandas as gpd
import pandas as pd

import folium
from IPython.display import display


In [15]:
# Load the shapefile
shapefile_path = "C:\\Users\\lukas\\OneDrive - Imperial College London\\0_Thesis\\1data_qgis\\Laos_Provinces\\provinces.shp"
gdf = gpd.read_file(shapefile_path)

# Calculate centroids manually
gdf['centroid'] = gdf.geometry.apply(lambda x: x.representative_point())

# Extract the province names and centroids
province_data = {
    'Province': gdf['NAME_1'],  # Replace 'Name' with the actual column name for province names if different
    'Latitude': gdf.centroid.y,
    'Longitude': gdf.centroid.x
}

# Create a DataFrame
df = pd.DataFrame(province_data)

# Save to Excel
file_path = 'Laos_Provinces_Centroids.xlsx'
df.to_excel(file_path, index=False)


C:\Users\lukas\AppData\Local\Temp\ipykernel_14060\1165236931.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  'Latitude': gdf.centroid.y,
C:\Users\lukas\AppData\Local\Temp\ipykernel_14060\1165236931.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  'Longitude': gdf.centroid.x


In [17]:
# Load the Excel file
file_path = 'Laos_Provinces_Centroids.xlsx'
df = pd.read_excel(file_path)

# Create a map centered around Laos
laos_map = folium.Map(location=[18.2, 105.3], zoom_start=7)

# Add markers for each province
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Province'],
        icon=folium.Icon(icon='info-sign')
    ).add_to(laos_map)

# Save the map to an HTML file
# laos_map.save('Laos_Provinces_Map.html')

# Display the map in the notebook
display(laos_map)

In [20]:
provinces = pd.read_excel('Laos_Provinces_Centroids.xlsx')
provinces

,Province,Area,Latitude,Longitude
0,Attapeu,south,14.793855,106.945076
1,Bokeo,north,20.351056,100.642907
2,Bolikhamxai,centre,18.488858,104.259675
3,Champasak,south,14.771686,105.960564
4,Houaphanh,north,20.283857,104.009343
5,Khammouane,centre,17.569541,105.260559
6,Luang Namtha,north,20.888413,101.143448
7,Luang Prabang,north,20.132591,102.552408
8,Oudomxay,north,20.418031,101.729691
9,Phongsaly,north,21.654447,102.241823


In [22]:
rice_production = pd.read_csv('FAO_rice_production_2022.csv')
rice_production = rice_production[['Province','harvested_2022','yield_2022','production_2022','production_average']]
rice_production.iloc[:3]

,Province,harvested_2022,yield_2022,production_2022,production_average
0,North,165,3.0,583,-17
1,Phongsaly,16,3.0,53,-22
2,Luang Namtha,20,3.0,52,13


In [23]:
demand_province = pd.merge(provinces, rice_production, on='Province', how='inner')
demand_province

,Province,Area,Latitude,Longitude,harvested_2022,yield_2022,production_2022,production_average
0,Attapeu,south,14.793855,106.945076,22,4.0,88,9
1,Bokeo,north,20.351056,100.642907,14,4.0,63,-30
2,Bolikhamxai,centre,18.488858,104.259675,40,4.0,165,3
3,Champasak,south,14.771686,105.960564,106,4.0,464,-4
4,Houaphanh,north,20.283857,104.009343,14,3.0,65,-40
5,Khammouane,centre,17.569541,105.260559,85,4.0,322,23
6,Luang Namtha,north,20.888413,101.143448,20,3.0,52,13
7,Luang Prabang,north,20.132591,102.552408,38,3.0,100,4
8,Oudomxay,north,20.418031,101.729691,20,3.0,87,-23
9,Phongsaly,north,21.654447,102.241823,16,3.0,53,-22


In [24]:
sum_production_2022 = demand_province.production_2022.sum()
demand_province['weighted_avg'] = demand_province.production_2022 / sum_production_2022

In [25]:
hydrogen_demand = 200000
demand_province['demand_hydrogen'] = hydrogen_demand * demand_province['weighted_avg']
demand_province

,Province,Area,Latitude,Longitude,harvested_2022,yield_2022,production_2022,production_average,weighted_avg,demand_hydrogen
0,Attapeu,south,14.793855,106.945076,22,4.0,88,9,0.026895,5378.973105
1,Bokeo,north,20.351056,100.642907,14,4.0,63,-30,0.019254,3850.855746
2,Bolikhamxai,centre,18.488858,104.259675,40,4.0,165,3,0.050428,10085.574572
3,Champasak,south,14.771686,105.960564,106,4.0,464,-4,0.141809,28361.858191
4,Houaphanh,north,20.283857,104.009343,14,3.0,65,-40,0.019866,3973.105134
5,Khammouane,centre,17.569541,105.260559,85,4.0,322,23,0.098411,19682.151589
6,Luang Namtha,north,20.888413,101.143448,20,3.0,52,13,0.015892,3178.484108
7,Luang Prabang,north,20.132591,102.552408,38,3.0,100,4,0.030562,6112.469438
8,Oudomxay,north,20.418031,101.729691,20,3.0,87,-23,0.026589,5317.848411
9,Phongsaly,north,21.654447,102.241823,16,3.0,53,-22,0.016198,3239.608802


In [26]:
demand_province

,Province,Area,Latitude,Longitude,harvested_2022,yield_2022,production_2022,production_average,weighted_avg,demand_hydrogen
0,Attapeu,south,14.793855,106.945076,22,4.0,88,9,0.026895,5378.973105
1,Bokeo,north,20.351056,100.642907,14,4.0,63,-30,0.019254,3850.855746
2,Bolikhamxai,centre,18.488858,104.259675,40,4.0,165,3,0.050428,10085.574572
3,Champasak,south,14.771686,105.960564,106,4.0,464,-4,0.141809,28361.858191
4,Houaphanh,north,20.283857,104.009343,14,3.0,65,-40,0.019866,3973.105134
5,Khammouane,centre,17.569541,105.260559,85,4.0,322,23,0.098411,19682.151589
6,Luang Namtha,north,20.888413,101.143448,20,3.0,52,13,0.015892,3178.484108
7,Luang Prabang,north,20.132591,102.552408,38,3.0,100,4,0.030562,6112.469438
8,Oudomxay,north,20.418031,101.729691,20,3.0,87,-23,0.026589,5317.848411
9,Phongsaly,north,21.654447,102.241823,16,3.0,53,-22,0.016198,3239.608802


In [35]:
demand_province[demand_province['Area'] == 'north'].demand_hydrogen.sum()

35635.69682151589